In [1]:
# dependencies

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import plotly.express as px
import datetime

from sqlalchemy import create_engine

pd.set_option('display.max_columns', None)

In [2]:
# Excel export of 'Invoice Detail' report

FILE_PATH = r"C:\Users\mikej\Downloads\SHOP Supplier Reporting - Invoice Details Excel - 2024-01-11T082933.128.xlsx"

# new_dot_data = FILE_PATH
# new_dot_data = r"C:\Users\mikej\Downloads\SHOP Supplier Reporting - Invoice Details Excel - 2024-01-08T145315.837.xlsx"

new_dot_data = pd.read_excel(FILE_PATH, header=2)
print(f"Oldest Date to Add")
print(f"{new_dot_data['Invoice Date'].min()}")
print(" ")
print(f"Most Recent Date to Add")
print(f"{new_dot_data['Invoice Date'].max()}")
new_dot_data.columns

Oldest Date to Add
2024-01-09 00:00:00
 
Most Recent Date to Add
2024-01-10 00:00:00


Index(['Supplier Name', 'Product Line Number', 'Product Line Description',
       'Buying Group Number', 'Buying Group Name', 'Customer #',
       'Customer Name', 'Customer Shipping City', 'Customer Shipping State',
       'Customer Shipping Zip Code', 'Customer Invoice Number', 'Invoice Date',
       'Customer PO Number', 'Customer Order Number', 'Dot #', 'MFG #',
       'Item UPC', 'Item Full Description', 'Qty Ordered', 'Qty Received',
       'Dollars', 'Customer Extended Gross Weight',
       'Customer Extended Net Weight', 'Channel Number', 'Channel Description',
       'Segment Code', 'Segment Description', 'Tier Number',
       'Tier Description', 'Product Line Sub-Category', 'Dot DC'],
      dtype='object')

In [3]:
# replace obove columns with...

new_dot_data.columns = ['supplier_name', 'product_line_number', 'product_line_desc',
       'buying_group_number', 'buying_group_name', 'customer_num',
       'customer_name', 'customer_shipping_city', 'customer_shipping_state',
       'customer_shipping_zip', 'customer_invoice_number', 'invoice_date',
       'customer_po_number', 'customer_order_number', 'prod_dot_number',
       'prod_mfg_number', 'item_upc', 'item_full_desc', 'qty_ordered',
       'qty_received', 'dollars', 'cust_ext_gross_weight',
       'cust_ext_net_weight', 'channel_num', 'channel_desc', 'segment_code',
       'segment_desc', 'tier_num', 'tier_desc', 'prod_line_sub_cat', 'dot_dc']

In [4]:
# database connection

db_password = "UnitCircle42!"
db_user = "postgres"
db_name = "dot"
endpoint = "awakedb.cre3f7yk1unp.us-west-1.rds.amazonaws.com"

connection_string = f"postgresql://{db_user}:{db_password}@{endpoint}:5432/{db_name}"
engine = create_engine(connection_string)

In [36]:
print(f"{len(new_dot_data)} new records from dates: \n"
        f"{new_dot_data.invoice_date.unique().month.min()}/{new_dot_data.invoice_date.unique().day.min()} thru {new_dot_data.invoice_date.unique().month.max()}/{new_dot_data.invoice_date.unique().day.max()}")

40 new records from dates: 
1/9 thru 1/10


In [37]:
##  -- send new rows to invoice_detail database

print("Records sent to db should match above")
new_dot_data.to_sql('invoice_detail', engine, if_exists='append', index=False)

Records sent to db should match above


40

In [ ]:
## read in invoice_detail for cleaning

dot = pd.read_sql('SELECT * FROM invoice_detail;', con = engine)
print(f"{len(dot)} rows")
dot.invoice_date.max()

In [ ]:
dot.columns

In [ ]:
dot.customer_name.nunique()

In [ ]:
# remove uneeded columns

dot.drop(columns=['supplier_name','product_line_number','product_line_desc', 'buying_group_name','buying_group_number','customer_num', 'customer_shipping_city', 'customer_shipping_state', 'customer_shipping_zip', 'prod_mfg_number', 'customer_invoice_number','customer_po_number','customer_order_number','prod_dot_number','item_upc','qty_ordered','cust_ext_gross_weight','cust_ext_net_weight', 'channel_desc', 'channel_num','segment_code','tier_num','tier_desc','prod_line_sub_cat','dot_dc'],inplace=True)

## add canadian dollars column
dot['cad'] = dot.dollars*1.33

## add column to denote the dot table
dot['sale_origin'] = 'dot'

# add parent distributor column
parent_child_table = pd.read_excel(r'C:\Users\mikej\Master Data\cpg-sales\data\parent_child.xlsx')

temp_dot = dot.set_index('customer_name')
parent_child_table = parent_child_table.set_index('customer_name')
dot = pd.merge(temp_dot,parent_child_table, how='left',left_index=True,right_index=True).reset_index()

dot.parent_customer.fillna(dot['customer_name'], inplace=True)

## CREATE CLEAN 'MARKET SEGMENT' COLUMN
tropicals = ['TROPICAL FOODS--DC', 'TROPICAL FOODS--FL', 'TROPICAL FOODS--GA',
       'TROPICAL FOODS--NV', 'TROPICAL FOODS--TN', 'TROPICAL FOODS--TX',
       'TROPICAL NUT & FRUIT']

other_segments = ['FOOD PROCESSORS','JAN/SAN/PAPER','ONLINE DISTRIBUTORS', 'E&S DEALERS','SYSTEMS']

def define_seg(row):    
    """duplicate segment description column changing
        tropical to alternative retail
    """
    if any(x in row['customer_name'] for x in tropicals): 
        return 'ALTERNATIVE RETAIL'
    elif any(x in row['customer_name'] for x in other_segments): 
        return 'OTHER' 
    else:
        return row['segment_desc']

dot['market_segment'] = dot.apply(define_seg, axis=1)


# add year/month columns
dot['invoice_date'] = pd.to_datetime(dot['invoice_date'])
year_col = dot.set_index(['invoice_date']).index.year#.astype('category')
month_col = dot.set_index(['invoice_date']).index.month_name()#.astype('category')

# month & year columns
dot.insert(0,"month", month_col)
dot.insert(1,"year", year_col)

# final fix dtypes
dot = dot.convert_dtypes()
dot['qty_received'] = dot['qty_received'].astype('float')

dot.drop(columns='segment_desc',inplace=True)

invoice_clean = dot.copy()

In [ ]:
# See what ya got

print(f"Earliest Date: {invoice_clean.invoice_date.min()}")
print(f"Latest Date:   {invoice_clean.invoice_date.max()}")
print(f"Rows: {len(invoice_clean)}")
print(" ")
invoice_clean.info()

In [ ]:
## PUSH TO CLEAN DOT DATABASE

invoice_clean.to_sql('invoice_clean', engine, if_exists='replace', index=False)

In [ ]:
# graph_df = pd.read_sql('SELECT * FROM invoice_clean;', con = engine)
# graph_df.invoice_date = pd.to_datetime(graph_df.invoice_date)

px.bar(invoice_clean.set_index('invoice_date').groupby(pd.Grouper(freq='Q'))['dollars'].sum(),
        y='dollars',
        template='presentation',
        labels={'dollars':'$USD',
                'invoice_date':''},
        color='dollars',
        width=900,
        height=400)

In [ ]:
## Read in Unleashed table from postgres

unl = pd.read_sql('SELECT * FROM unl;', con = engine)
unl['USD'] = unl['sub_total']*.75

unl['completed_date'] = pd.to_datetime(unl['completed_date'])

print(unl.completed_date.min())
print(unl.completed_date.max())
print(f"Rows: {len(unl[unl['completed_date'] > '2022'])}")

In [ ]:
unl.columns

In [ ]:
# Dot Outbounds

df = (
    dot
    .groupby('invoice_date',as_index=False)
    ['dollars'].sum()
)

df.index = pd.to_datetime(df['invoice_date'],format='%m/%d/%y %I:%M%p')
df = pd.DataFrame(df.groupby(by=[df.index.year, df.index.month],as_index=False)['dollars'].sum())

px.scatter(y=df.dollars,
        trendline="rolling", trendline_options=dict(function="median", window=5), trendline_scope="overall",
        title='Dot Outbounds',
        template='presentation',
        opacity=.2,
        labels={'y':'$USD',
                'x':'Date'
               }
       )

In [ ]:
px.scatter(dot.groupby('invoice_date',as_index=False)['dollars'].sum(),
       x='invoice_date',
       y='dollars',
#        trendline='lowess',
       trendline="rolling", trendline_options=dict(function="median", window=20), trendline_scope="overall",    
       trendline_color_override="red",
#        template='presentation',
       color='dollars',
        opacity=.2)

In [ ]:
## turns dot to raw_dot_outbounds (TABLEAU READY)

# Update columns headers to match original raw data (for tableau)

labels = ['Supplier Name','Product Line Number', 'Product Line Description','Buying Group Number', 'Buying Group Name', 'Customer #',
            'Customer Name','Customer Shipping City','Customer Shipping State','Customer Shipping Zip Code',
            'Customer Invoice Number','Invoice Date','Customer PO Number','Customer Order Number','Dot #',
             'MFG #','Item UPC','Item Full Description','Qty Ordered','Qty Received','Dollars','Customer Extended Gross Weight',
             'Customer Extended Net Weight','Channel Number','Channel Description','Segment Code','Segment Description',
            'Tier Number','Tier Description','Product Line Sub-Category','Dot DC','Table','Segment Description 2','Parent Customer']

raw_dot_outbounds = dot.copy()
raw_dot_outbounds.columns = labels

# raw_dot_outbounds.head(2)